# Processing the CoreLogic Data using PySpark

In this Jupyter notebook, we demonstrate how you can process the CoreLogic data using PySpark. 

In particular, we will show you how to:
* import the data, 
* explore rows and columns, 
* filter for a given location, 
* inspect the data for missing and erroneous information, 
* correct data errors,
* and write a subset of the data to a csv file for later use. 

This [PySpark cheat sheet](http://datacamp-community-prod.s3.amazonaws.com/acfa4325-1d43-4542-8ce4-bea2d287db10) provides a great overview of available PySpark functionality. 

The notebook was created using a 'Jupyter + Spark Basic session' in [Open on Demand](https://arc.umich.edu/open-ondemand/) (OOD) on [Great Lakes](https://arc.umich.edu/greatlakes/) (GL). This automatically initializes Spark in the background.

If you are not running the notebook using OOD on GL, you will most likely have to make sure PySpark is installed on your system and then initialize it by typing 

```from pyspark import SparkContext```

```sc = SparkContext(master = 'local[2]')```

If you encounter any errors or have questions about this Jupyter notebook, or if you would like us to add another PySpark example using the CoreLogic data, feel free to reach out to [Armand Burks](mailto:arburks@umich.edu) and [Jule Krüger](mailto:julianek@umich.edu).

## Importing the CoreLogic data with PySpark

The CoreLogic data are stored in a Turbo volume on the ```/nfs``` drive. To get access, you will need to sign a Memorandum of Understanding (MOU) with the University of Michigan Library. For more information about this, see [here](https://github.com/arc-ts/corelogic-on-greatlakes/tree/main/intro-to-corelogic-data). To execute this notebook, you will have to be granted access to the CoreLogic data on Turbo.

The raw data come in three separate files: deeds (28GB), foreclosures (6GB), and taxes (24GB). [CSCAR](https://cscar.research.umich.edu/) pre-processed the CoreLogic data and stored each raw file in 100 separate partitions. You can read more about this methodology in ```nfs/turbo/lib-data-corelogic/Docs/cscar_data.txt```.

We are going to work with the pre-processed data to improve import speeds. Let's store the paths to the pre-processed partitioned files in three separate variables:

In [1]:
turbo_path_fcl = "/nfs/turbo/lib-data-corelogic/Data/fcl/*.gz"
turbo_path_deed = "/nfs/turbo/lib-data-corelogic/Data/deed/*.gz"
turbo_path_tax = "/nfs/turbo/lib-data-corelogic/Data/tax/*.gz"

The following command imports a set of partitioned CoreLogic data file into a dataframe (df). You can switch between foreclosures, deeds, and taxes by choosing the relevant path accordingly. In each text file, the pipe ```|``` was used as a delimiter and is set  in the ```sep``` argument as such.

__A note on import speeds__: Because of their size, it takes a little while to read in each partitioned data file collection. Importing the raw data files takes much longer. Using the pre-processed data from CSCAR, which distributes the data across 100 partitions, greatly improves import speeds as Spark can spread computation across multiple cores.

Let's work with the deed data for now:

In [7]:
df = spark.read.csv(turbo_path_deed, header=True, sep="|")

(A small note on reading in the data: You could set the ```inferSchema=True``` argument, which would make Python infer the column type. The drawback of this is that FIPS and ZIP codes that start with a `0` have the leading `0` removed in the creation of an integer variable. We choose to keep ```inferSchema=False```, the default argumentn to avoid this.)

## Exploring the CoreLogic data

First, we want to get a basic idea of the CoreLogic data. What are the columns and what types of values do they contain? The .printSchema() method prints the data type (integer, string, double) for each column in the dataset. 

Note that the ```double``` data type is unique to Spark. It represents double precision floats [[source](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.DoubleType.html)]. Also, ```nullable = true``` means that a given column can accept missing (```null```) values.

In [8]:
df.printSchema()

root
 |-- FIPS: string (nullable = true)
 |-- APN (Parcel Number) (unformatted): string (nullable = true)
 |-- PCL ID IRIS FORMATTED: string (nullable = true)
 |-- APN SEQUENCE NUMBER: string (nullable = true)
 |-- PENDING RECORD INDICATOR: string (nullable = true)
 |-- CORPORATE INDICATOR: string (nullable = true)
 |-- OWNER FULL NAME: string (nullable = true)
 |-- OWNER 1 LAST NAME: string (nullable = true)
 |-- OWNER 1 FIRST NAME & M I: string (nullable = true)
 |-- OWNER 2 LAST NAME: string (nullable = true)
 |-- OWNER 2 FIRST NAME & MI: string (nullable = true)
 |-- OWNER ETAL INDICATOR: string (nullable = true)
 |-- C/O NAME: string (nullable = true)
 |-- OWNER RELATIONSHIP RIGHTS CODE: string (nullable = true)
 |-- OWNER RELATIONSHIP TYPE: string (nullable = true)
 |-- PARTIAL INTEREST INDICATOR: string (nullable = true)
 |-- ABSENTEE OWNER STATUS: string (nullable = true)
 |-- PROPERTY LEVEL LATITUDE: string (nullable = true)
 |-- PROPERTY LEVEL LONGITUDE: string (nullable = tr

We can calculate how many rows and columns there are in total.

In [4]:
print((df.count(), len(df.columns))) ## print number of rows and columns

(367782480, 97)


We can print a list of the column names like so.

In [5]:
df.columns

['FIPS',
 'APN (Parcel Number) (unformatted)',
 'PCL ID IRIS FORMATTED',
 'APN SEQUENCE NUMBER',
 'PENDING RECORD INDICATOR',
 'CORPORATE INDICATOR',
 'OWNER FULL NAME',
 'OWNER 1 LAST NAME',
 'OWNER 1 FIRST NAME & M I',
 'OWNER 2 LAST NAME',
 'OWNER 2 FIRST NAME & MI',
 'OWNER ETAL INDICATOR',
 'C/O NAME',
 'OWNER RELATIONSHIP RIGHTS CODE',
 'OWNER RELATIONSHIP TYPE',
 'PARTIAL INTEREST INDICATOR',
 'ABSENTEE OWNER STATUS',
 'PROPERTY LEVEL LATITUDE',
 'PROPERTY LEVEL LONGITUDE',
 'SITUS HOUSE NUMBER PREFIX',
 'SITUS HOUSE NUMBER',
 'SITUS HOUSE NUMBER SUFFIX',
 'SITUS DIRECTION',
 'SITUS STREET NAME',
 'SITUS MODE',
 'SITUS QUADRANT',
 'SITUS APARTMENT UNIT',
 'SITUS CITY',
 'SITUS STATE',
 'SITUS ZIP CODE',
 'SITUS CARRIER CODE',
 'MAILING HOUSE NUMBER PREFIX',
 'MAILING HOUSE NUMBER',
 'MAILING HOUSE NUMBER SUFFIX',
 'MAILING DIRECTION',
 'MAILING STREET NAME',
 'MAILING MODE',
 'MAILING QUADRANT',
 'MAILING APARTMENT UNIT',
 'MAILING PROPERTY CITY',
 'MAILING PROPERTY STATE',
 'MA

To better understand the columns, we can print summary statistics for those of type ```integer``` and ```double```.

The 'FIPS' column designates the Federal Information Processing Standard county code. It contains five digits. As we can see in the summary statistics below, the 'FIPS' column contains errors as the minimum value of 1001 only contains four digits.

In [7]:
df.describe(['FIPS']).show()

+-------+------------------+
|summary|              FIPS|
+-------+------------------+
|  count|         367782480|
|   mean| 25295.95461476577|
| stddev|16730.499490625767|
|    min|              1001|
|    max|             97201|
+-------+------------------+



A statistical summary of the FIPS column is not meaningful as it represents a discrete/categorical variable (ordinal scale of measurement). 

Let's look at a column of type ```double```. The 'MORTGAGE AMOUNT' represents a continuous variable. As you can see, the average mortgage in the CoreLogic data was about $800K. Some properties had no mortgage taken on them (minimum value = 0). The largest mortgage (maximum value) was USD 33 billion 300 million.

In [8]:
df.describe(['MORTGAGE AMOUNT']).show()

+-------+-------------------+
|summary|    MORTGAGE AMOUNT|
+-------+-------------------+
|  count|          111059401|
|   mean|  799771.0156008194|
| stddev|2.141949398883503E7|
|    min|                0.0|
|    max|            3.33E10|
+-------+-------------------+



Let's look at one more variable of interest, the sale amount of a property. This variable is interesting because it is stored as a ```string```. While it should be numerical, this means it must contain some type of non-numerical values. As we can see below, the maximum value is 'U'. When we move to a data cleaning step, we would probably want to set non-numerical values in this column to missing (```null```).

In [11]:
df.describe(['SALE AMOUNT']).show()

+-------+--------------------+
|summary|         SALE AMOUNT|
+-------+--------------------+
|  count|           213239914|
|   mean|    418105.188683447|
| stddev|2.5857146238130517E7|
|    min|                 .01|
|    max|                   U|
+-------+--------------------+



We might also be interested in the range of unique values of categorical variables. Let's print the unique values of states designated by the 'SITUS STATE' column.

In [12]:
len(df.select('SITUS STATE').distinct().rdd.map(lambda r: r[0]).collect())

53

The SITUS STATE column contains 53 unique values, which is unexpected. There should be 50 states plus Washington, DC. Let's create a frequency table of the column values to get a better idea about this variable. We will sort the table in descending order of state counts. We will set ```n=53``` in the show() method to print all rows in the table. The two additional values are ```null``` (the amount of missing values in the SITUS STATE column). The additional territory captured as a state here is VI, the Virgin Islands.

TODO: sort this column by count in ascending or descending order

In [23]:
df.groupBy('SITUS STATE').count().show(n=53)

+-----------+--------+
|SITUS STATE|   count|
+-----------+--------+
|         SC| 6311502|
|         AZ|12006118|
|         LA| 2497956|
|         MN| 4267855|
|         NJ| 7320302|
|         DC|  457645|
|         OR| 4767128|
|         VA| 7827533|
|       null|35371569|
|         RI|  465933|
|         WY|  300540|
|         KY| 3018343|
|         NH|  559467|
|         MI| 8638900|
|         NV| 4712444|
|         WI| 3864221|
|         ID| 1138025|
|         CA|52167596|
|         CT| 2513164|
|         NE| 1365483|
|         MT| 1289606|
|         NC| 9686809|
|         VT|  329881|
|         MD| 6525203|
|         DE|  905431|
|         MO| 6496410|
|         VI|   20620|
|         IL|11947950|
|         ME|  614182|
|         WA| 8104844|
|         ND|  426858|
|         MS| 2978654|
|         AL| 4379161|
|         IN| 4394358|
|         OH|13347704|
|         TN| 8638649|
|         IA| 1982068|
|         NM| 1546001|
|         PA|12254911|
|         SD|  318106|
|         N

## What amount of information is missing in the CoreLogic data?

It is important to know for columns of interest how many missing values they contain. Information on data missingness guides decisions on what needs to be done with affected rows. Can rows with missing information easily be dropped? Or, do we need to try to fill in missing values conditional on information in other columns?

Suppose we wanted to filter the CoreLogic data based on location. Our goal will be to create a subset of the data that only contains rows relating to Detroit, Michigan. 

Let's print a select few rows and columns of the dataframe.

In [9]:
df.select('FIPS', 'SITUS CITY', 'SITUS STATE', 'SITUS ZIP CODE').show()

+-----+----------+-----------+--------------+
| FIPS|SITUS CITY|SITUS STATE|SITUS ZIP CODE|
+-----+----------+-----------+--------------+
|12099|      null|       null|          null|
|17097|      null|       null|          null|
|97200|      null|       null|          null|
|97200|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|17097|      null|       null|    

Notice how there are many rows containing columns with null values. We will remove rows that hvae nulls in the columns of interest for the later analysis below.

In [6]:
df = df.dropna(subset=["FIPS","SITUS CITY", "SITUS STATE", "SITUS ZIP CODE"])

Now let's see how many rows we have out of the original 367,782,480:

In [7]:
df.count()

311359183

Let's assume we wanted to filter the CoreLogic data only for Wayne county, MI. The FIPS code for Wayne county is '26163' ([source](https://mi.postcodebase.com/county/26163)).

In [8]:
WayneCsubset = df.filter(df.FIPS==26163)

Let's print a few rows and columns of the subset. As you can see, we are only dealing with Wayne county now.

In [9]:
WayneCsubset.select('FIPS', 'SITUS CITY', 'SITUS STATE', 'SITUS ZIP CODE').show()

+-----+----------+-----------+--------------+
| FIPS|SITUS CITY|SITUS STATE|SITUS ZIP CODE|
+-----+----------+-----------+--------------+
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482012463|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|     482021302|
|26163|   DETROIT|         MI|     482021368|
|26163|   DETROIT|         MI|     482021368|
|26163|   DETROIT|         MI|     482021368|
|26163|   DETROIT|         MI|    

How many rows are in the Wayne county subset? Use the count() method

In [10]:
WayneCsubset.count()

2132959

There are many different cities within Wayne county. Let's get a list of all of them:

In [11]:
WayneCsubset.select('SITUS CITY').distinct().collect()

[Row(SITUS CITY='LINCOLN PARK'),
 Row(SITUS CITY='MELVINDALE'),
 Row(SITUS CITY='NORTHVILLE'),
 Row(SITUS CITY='TRENTON'),
 Row(SITUS CITY='YPSILANTI'),
 Row(SITUS CITY='BELLEVILLE'),
 Row(SITUS CITY='OAK PARK'),
 Row(SITUS CITY='RIVERVIEW'),
 Row(SITUS CITY='GROSSE POINTE WOODS'),
 Row(SITUS CITY='FLAT ROCK'),
 Row(SITUS CITY='GROSSE ILE'),
 Row(SITUS CITY='GROSSE POINTE PARK'),
 Row(SITUS CITY='WARREN'),
 Row(SITUS CITY='GIBRALTAR'),
 Row(SITUS CITY='GROSSE POINTE'),
 Row(SITUS CITY='MONROE'),
 Row(SITUS CITY='WYANDOTTE'),
 Row(SITUS CITY='HIGHLAND'),
 Row(SITUS CITY='WOODHAVEN'),
 Row(SITUS CITY='INKSTER'),
 Row(SITUS CITY='SOUTHGATE'),
 Row(SITUS CITY='SUMPTER TWP'),
 Row(SITUS CITY='GARDEN CITY'),
 Row(SITUS CITY='HIGHLAND PARK'),
 Row(SITUS CITY='DETROIT'),
 Row(SITUS CITY='FERNDALE'),
 Row(SITUS CITY='CHELSEA'),
 Row(SITUS CITY='BROWNSTOWN TWP'),
 Row(SITUS CITY='NEW HUDSON'),
 Row(SITUS CITY='LANSING'),
 Row(SITUS CITY='DEARBORN HEIGHTS'),
 Row(SITUS CITY='CANTON'),
 Row(SITUS 

Let's create a subset for Detroit only. We will filter using the 'SITUS CITY' variable. This variables designates the city associated with the property address. 

Note that this is a straightforward filtering method if using the Wayne county subset we created previously. 

In [12]:
detroit = WayneCsubset.filter(WayneCsubset["SITUS CITY"]=="DETROIT")

If you wanted to create a Detroit subset from the entire dataset, we suggest filtering on 'SITUS CITY' and 'SITUS STATE'. When using the entire dataframe, filtering on 'SITUS CITY' alone could run into city name ambiguity across states. 

In [13]:
detroit2 = df.filter((df["SITUS CITY"]=="DETROIT") & (df["SITUS STATE"]=="MI"))

How many rows are in the Detroit subset? Depending on the size of the dataframe, count() can take a bit of time to calculate. Therefore, we will save the value to a variable in case we need to use it later on. This can save us some processing time.

In [16]:
detroitCount = detroit.count()
detroitCount

1129407

Do the two filtering methods yield the same number of rows?

In [17]:
detroit2Count = detroit2.count()
detroit2Count

1129438

We see that we get 2 different results when we filter by (SITUS CITY == "DETROIT" and SITUS STATE == "MI") versus (FIPS == 26163 and SITUS CITY == "DETROIT"). This is because there are multiple FIPS codes where the SITUS CITY is equal to Detroit. Let's see all the FIPS codes for rows with city and state Detroit, MI.

In [18]:
detroit2.groupBy('FIPS', 'SITUS CITY', 'SITUS STATE').count().show()

+-----+----------+-----------+-------+
| FIPS|SITUS CITY|SITUS STATE|  count|
+-----+----------+-----------+-------+
|26147|   DETROIT|         MI|      6|
|26071|   DETROIT|         MI|      5|
|26163|   DETROIT|         MI|1129407|
|26061|   DETROIT|         MI|      2|
|48039|   DETROIT|         MI|      2|
|26069|   DETROIT|         MI|      2|
|26035|   DETROIT|         MI|      2|
|26115|   DETROIT|         MI|      9|
|26099|   DETROIT|         MI|      2|
|26161|   DETROIT|         MI|      1|
+-----+----------+-----------+-------+



The detroit2 dataframe looks like what we are interested in, as it has all the rows from Detroit, MI. Let's write the Detroit data to a new csv file in an ```output/``` folder in our ```home/[uniqname]/``` directory. (Note: You probably need to change this path, depending on where you store this notebook.) By default, Spark writes big data into multiple files to optimize computation times during import and export. This is recommended practice, especially if the data is big, i.e., if it has many rows and/or columns.

The below command writes 100 partitions of the Detroit data to the designated folder. 

_Note_: If you saved data to this path before, you will need to add the argument ```overwrite=True``` to the function call.

In [15]:
detroit2.write.csv("../../output/corelogic_data_deeds_Detroit_partitioned")

There is also a way to force the data into one single file. Sometimes, a single file is needed to continue processing the data with other research software. Saving the data into one file is a little risky and might not always work, so caution is advised. 

The following command combines the data into one core (Caution: it is very slow). If the data does not fit, it will throw an error. _Note_: If you saved data to this path before, you will need to add the argument ```overwrite=True``` to the function call. The overwrite argument would overwrite the previously created ```output/``` directory.

Note: You would only need to run this, if you indeed need only one single data file. Many research programs allow you to read in and combine multiple data files into one object. See for example the [glob method in Python](https://docs.python.org/3/library/glob.html).

In [ ]:
detroit2.coalesce(1).write.csv("../../output/corelogic_data_deeds_Detroit_singlefile")

Let's say we want to draw and save just a sample of the Detroit data (n~1000) that is small enough to use for writing and testing code for analyzing the Detroit subset. We can use PySpark's DataFrame.sample() method. However, we have to specify a fraction of the dataframe size rather than explicitly the size of the sample, n. Also, we are not guaranteed to get EXACTLY the fraction that we specify, due to the way the sample() function is implemented. Let's see how that works.

Since we want about 1000 rows in our sample, we can divide 1000 by detroit2Count to see what fraction that is.

In [19]:
sampleFraction = 1000 / detroit2Count
sampleFraction

0.0008853960996531018

Now, let's sample the Detroit data.

In [20]:
detroitSample = detroit2.sample(fraction=sampleFraction)

How many records do we have in our sample?

In [21]:
detroitSample.count()

1031

Write the results (note that using coalesce(1) can still be quite expensive. Use only if absolutely necessary).

In [23]:
detroitSample.write.csv("../../output/corelogic_data_deeds_Detroit_sampled_1031_singlefile")